<img src="https://climatecompatiblegrowth.com/wp-content/uploads/OnSSET_text.png" height="150" align="center">

<img src="https://avatars.githubusercontent.com/u/62061836?s=200&v=4" height="80" align="center">

<img src="https://onstove-documentation.readthedocs.io/en/latest/_images/kth_logo.svg" height="80" align="center">

<img src="https://www.ukaiddirect.org/images/logo.png" height="70" align="center">

# Welcome to the Starter Data Kit for `OnSSET`!
This notebook will help you to download the starter datasets needed to run a basic `OnSSET` analysis. It will allow you to download the geospatial data required as well as the socio- and techno-economic data of your country of interest.

Please follow the steps below and choose the country and datasets to download.

In [ ]:
#%%capture
# @title Setup {"vertical-output":true,"display-mode":"form"}
# @markdown Run this cell for setting-up the connection to the database.
# Install boto3 and localtileserver if not already installed

import subprocess
from IPython.display import clear_output

def run_quiet_shell(command):
    # Start the process
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

    # Read output line by line as it happens
    for line in process.stdout:
        clear_output(wait=True)
        print(line.strip())

    process.wait()

# Usage:
run_quiet_shell("pip install --ignore-installed blinker tqdm --upgrade")
run_quiet_shell("pip install git+https://github.com/ClimateCompatibleGrowth/Starter-Data-Kits.git --upgrade")
run_quiet_shell("pip install ipyleaflet leafmap localtileserver --upgrade")

from google.colab import userdata
import os
# import awswrangler as wr
import ipywidgets as widgets
from ipywidgets import Dropdown, Layout, SelectMultiple, Button, HBox, VBox
from IPython.display import display
from google.colab.output import eval_js
from IPython.display import HTML
import starterkits as sdk
import geopandas as gpd
from tqdm.notebook import tqdm
import glob

print('Setup complete')

In [ ]:
# @title Select datasets to download {"run":"auto","vertical-output":true}
# @markdown Select the country to download data for:

# all road types
all_road_types = [
    "motorway", "motorway_link",
    "trunk", "trunk_link",
    "primary", "primary_link",
    "secondary", "secondary_link",
    "tertiary", "tertiary_link",
    "unclassified", "residential",
    "service", "living_street"
]

# Road types to keep
suggested_road_types = [
    "motorway", "motorway_link",
    "trunk", "trunk_link",
    "primary", "primary_link",
    "secondary", "secondary_link",
    "tertiary", "tertiary_link"
]


display(HTML(''' '''))

Country = "Select country" # @param ["Select country", 'AGO', 'BDI', 'BEN', 'BFA', 'BWA', 'CAF', 'CIV', 'CMR', 'COD', 'COG', 'DJI', 'ERI', 'ETH', 'GAB', 'GHA', 'GIN', 'GMB', 'GNB', 'GNQ', 'KEN', 'LBR', 'LSO', 'MDG', 'MLI', 'MOZ', 'MRT', 'MWI', 'NAM', 'NER', 'NGA', 'RWA', 'SDN', 'SEN', 'SLE', 'SOM', 'SSD', 'SWZ', 'TCD', 'TGO', 'TZA', 'UGA', 'ZAF', 'ZMB', 'ZWE']
# @markdown Choose the datasets to download in the optios below:

box_layout = widgets.Layout(display='flex',
                            flex_flow='column',
                            align_items='center',
                            width='100%')

# Create the widget
pop_checkbox = widgets.Checkbox(
    description='Population count', # Label
    value=False,                            # Default value
    disabled=False,                         # Whether to disable user changes
    indent=False                             # Align with other controls with a description
)

pop_dropdown = widgets.Dropdown(
    description='Dataset',
    options=['Worldpop'], # 'GHSL'
    disabled=True
)

pop_radio = widgets.RadioButtons(
    options=['100m', '1km'],
    description='Resolution:',
    disabled=True
)

pop_box = VBox([pop_checkbox, HBox([pop_dropdown, pop_radio])])

mv_checkbox = widgets.Checkbox(
    description='Medium voltage lines', # Label
    value=False,                            # Default value
    disabled=False,                         # Whether to disable user changes
    indent=False                             # Align with other controls with a description
)

mv_dropdown = widgets.Dropdown(
    description='Dataset',
    options=['GridFinder'], # 'energydata.info'
    disabled=True
)

mv_box = VBox([mv_checkbox, HBox([mv_dropdown])])

wind_checkbox = widgets.Checkbox(
    description='Wind speed', # Label
    value=False,                            # Default value
    disabled=False,                         # Whether to disable user changes
    indent=False                             # Align with other controls with a description
)

wind_dropdown = widgets.Dropdown(
    description='Dataset',
    options=['Global Wind Atlas'],
    disabled=True
)

wind_radio = widgets.RadioButtons(
    options=['100', '150', '200'],
    description='Height (m):',
    disabled=True
)

wind_box = VBox([wind_checkbox, HBox([wind_dropdown, wind_radio])])

solar_checkbox = widgets.Checkbox(
    description='Solar irradiation', # Label
    value=False,                            # Default value
    disabled=False,                         # Whether to disable user changes
    indent=False                             # Align with other controls with a description
)

solar_dropdown = widgets.Dropdown(
    description='Dataset',
    options=['Global Solar Atlas'],
    disabled=True
)

solar_box = VBox([solar_checkbox, HBox([solar_dropdown])])

specs_checkbox = widgets.Checkbox(
    description='Socio-economic specifications', # Label
    value=False,                            # Default value
    disabled=False,                         # Whether to disable user changes
    indent=False                             # Align with other controls with a description
)

dem_checkbox = widgets.Checkbox(
    description='Elevation model', # Label
    value=False,                            # Default value
    disabled=False,                         # Whether to disable user changes
    indent=False                             # Align with other controls with a description
)

dem_description = widgets.HTML('Digital Model Data is downloaded from the ' +
                               'OpenTopography project, for which you need an API key. ' +
                               'Please go to https://opentopography.org create a ' +
                               'free account and generate an API key. Copy the ' +
                               'and paste it in the field below.', layout=Layout(width='700px'))

dem_dropdown = widgets.Dropdown(
    description='Dataset',
    options=['NASADEM'],
    disabled=True
)

dem_api = widgets.Text(
    value='',
    placeholder='Enter API key',
    description='API key:',
    disabled=True
)

dem_box = VBox([dem_checkbox, dem_description, HBox([dem_dropdown, dem_api])])

land_checkbox = widgets.Checkbox(
    description='Land cover', # Label
    value=False,                            # Default value
    disabled=False,                         # Whether to disable user changes
    indent=False
)

land_description = widgets.HTML('Land Cover Data is downloaded from ' +
                               'Nasa Earth data, for which you need a user a password. ' +
                               'Please go to https://urs.earthdata.nasa.gov/home create a ' +
                               'free account and fill the login credentials.', layout=Layout(width='700px'))

land_dropdown = widgets.Dropdown(
    description='Dataset',
    options=['MODIS (MCD12Q1)'],
    disabled=True
)

land_user = widgets.Text(
    value='',
    placeholder='Enter username',
    description='Username:',
    disabled=True
)

land_password = widgets.Text(
    value='',
    placeholder='Enter password',
    description='password:',
    disabled=True
)

land_box = VBox([land_checkbox, land_description, HBox([land_dropdown,
                                                        VBox([land_user,
                                                              land_password])])])

ntl_checkbox = widgets.Checkbox(
    description='Nighttime lights', # Label
    value=False,                            # Default value
    disabled=False,                         # Whether to disable user changes
    indent=False                             # Align with other controls with a description
)

ntl_dropdown = widgets.Dropdown(
    description='Dataset',
    options=['WorldPop covariates derived from VNL 2.1/2.2'],
    disabled=True
)

ntl_box = VBox([ntl_checkbox, HBox([ntl_dropdown])])

roads_checkbox = widgets.Checkbox(
    description='Roads', # Label
    value=False,                            # Default value
    disabled=False,                         # Whether to disable user changes
    indent=False                             # Align with other controls with a description
)

roads_dropdown = widgets.Dropdown(
    description='Dataset',
    options=['OSM road network'],
    disabled=True
)

roads_types = widgets.SelectMultiple(
    options=all_road_types,
    value=suggested_road_types,
    rows=10,
    description='Road types',
    disabled=True
)

roads_box = VBox([roads_checkbox, HBox([roads_dropdown])]) # , roads_types])])

download = Button(description='Get data',
                  icon= 'download',
                  layout=Layout(width='200px', height='30px'),
                  button_style='primary')

def on_pop_checkbox_changed(i):
  if pop_checkbox.value:
    pop_dropdown.disabled = False
    pop_radio.disabled = False
  else:
    pop_dropdown.disabled = True
    pop_radio.disabled = True

def on_mv_checkbox_changed(i):
  if mv_checkbox.value:
    mv_dropdown.disabled = False
  else:
    mv_dropdown.disabled = True

def on_wind_checkbox_changed(i):
  if wind_checkbox.value:
    wind_dropdown.disabled = False
    wind_radio.disabled = False
  else:
    wind_dropdown.disabled = True
    wind_radio.disabled = True

def on_solar_checkbox_changed(i):
  if solar_checkbox.value:
    solar_dropdown.disabled = False
  else:
    solar_dropdown.disabled = True

def on_dem_checkbox_changed(i):
  if dem_checkbox.value:
    dem_dropdown.disabled = False
    dem_api.disabled = False
  else:
    dem_dropdown.disabled = True
    dem_api.disabled = True

def on_land_checkbox_changed(i):
  if land_checkbox.value:
    land_dropdown.disabled = False
    land_user.disabled = False
    land_password.disabled = False
  else:
    land_dropdown.disabled = True
    land_user.disabled = True
    land_password.disabled = True

def on_ntl_checkbox_changed(i):
  if ntl_checkbox.value:
    ntl_dropdown.disabled = False
  else:
    ntl_dropdown.disabled = True

def on_roads_checkbox_changed(i):
  if roads_checkbox.value:
    roads_dropdown.disabled = False
    roads_types.disabled = False
  else:
    roads_dropdown.disabled = True
    roads_types.disabled = True

def on_download_clicked(i):
  total_downloads = 1 # for boundaries

  if specs_checkbox.value:
    total_downloads += 1
  if pop_checkbox.value:
    total_downloads += 1
  if mv_checkbox.value:
    total_downloads += 1
  if wind_checkbox.value:
    total_downloads += 1
  if solar_checkbox.value:
    total_downloads += 1
  if dem_checkbox.value:
    total_downloads += 1
  if land_checkbox.value:
    total_downloads += 1
  if ntl_checkbox.value:
    total_downloads += 1
  if roads_checkbox.value:
    total_downloads += 1

  with tqdm(total=total_downloads, desc="Downloading datasets") as pbar:
    sdk.apis.get_boundaries(Country)
    pbar.update(1)
    pbar.set_description("Downloading boundaries")

    if specs_checkbox.value:
      sdk.apis.get_specs(Country)
      pbar.update(1)
      pbar.set_description("Downloading socio-economic specifications")
    if pop_checkbox.value:
      sdk.apis.get_population_data(Country, pop_radio.value)
      pbar.update(1)
      pbar.set_description("Downloading population data")
    if mv_checkbox.value:
      sdk.apis.get_power_lines(Country)
      pbar.update(1)
      pbar.set_description("Downloading medium voltage lines")
    if wind_checkbox.value:
      sdk.apis.get_wind_data(Country, wind_radio.value)
      pbar.update(1)
      pbar.set_description("Downloading wind speed data")
    if solar_checkbox.value:
      sdk.apis.get_solar_data(Country)
      pbar.update(1)
      pbar.set_description("Downloading solar irradiation data")
    if dem_checkbox.value:
      sdk.apis.get_dem_data(Country, dem_api.value)
      pbar.update(1)
      pbar.set_description("Downloading elevation model data")
    if land_checkbox.value:
      sdk.apis.get_landcover_data(country=Country, username=land_user.value,
                                  password=land_password.value)
      pbar.update(1)
      pbar.set_description("Downloading land cover data")
    if ntl_checkbox.value:
      sdk.apis.get_ntl_data(Country)
      pbar.update(1)
      pbar.set_description("Downloading nighttime lights data")
    if roads_checkbox.value:
      sdk.apis.get_roads(Country) # , roads_types.value)
      pbar.update(1)
      pbar.set_description("Downloading roads data")
    print('\033[1mData downloaded succesfully to session\033[0m')

hr = widgets.HTML(value="<hr>")
space = widgets.HTML(value="<br>")

pop_checkbox.observe(on_pop_checkbox_changed, names='value')
mv_checkbox.observe(on_mv_checkbox_changed, names='value')
wind_checkbox.observe(on_wind_checkbox_changed, names='value')
solar_checkbox.observe(on_solar_checkbox_changed, names='value')
dem_checkbox.observe(on_dem_checkbox_changed, names='value')
land_checkbox.observe(on_land_checkbox_changed, names='value')
ntl_checkbox.observe(on_ntl_checkbox_changed, names='value')
roads_checkbox.observe(on_roads_checkbox_changed, names='value')
download.on_click(on_download_clicked)

# Display the widget
if Country != 'Select country':
  display(VBox([specs_checkbox, hr, pop_box, hr, mv_box, hr, wind_box, hr,
                solar_box, hr, dem_box, hr, land_box, hr, ntl_box, hr, roads_box,
                space, download]))
else:
  print('Please select a country')


In [ ]:
# @title Delete country data {"run":"auto"}
# @markdown Run this cell to select a country to delete from the session.
# # delete data for a country
display(HTML('''<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css"> '''))

def on_delete(b):
  try:
    !rm -rf /content/Data/{country_dropdown.value}
  except:
    pass
  if country_dropdown.value is not None:
    print(f'{country_dropdown.value} data deleted!')
    country_dropdown.options = get_countries()

def get_countries():
  countries = set()
  try:
    countries.update([files.split('/')[1] for files in glob.glob(f'Data/*')])
  except:
    pass
  return countries


country_dropdown = widgets.Dropdown(
                description='Country',
                options=get_countries(),
                disabled=False
)
delete = Button(description='Delete data',
                icon='trash',
                layout=Layout(width='200px', height='35px'),
                button_style='danger')

delete.on_click(on_delete)

display(HBox([country_dropdown, delete]))

In [ ]:
# @title Download data
# @markdown Run this cell to download all files in the session to your computer.
import os
import zipfile
from google.colab import files as cfiles
import glob

def zip_and_download(folder_names, zip_filename="data.zip"):
  """Zips specified folders and downloads the zip file."""

  with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for folder_name in folder_names:
      for root, _, files in os.walk(folder_name):
        for file in files:
          file_path = os.path.join(root, file)
          arcname = os.path.relpath(file_path, '/content/') #Maintain folder structure
          zipf.write(file_path, arcname=arcname)

  cfiles.download(zip_filename)


# Specify the folders to be zipped
folders_to_zip = glob.glob('Data/*')
zip_and_download(folders_to_zip)